# Table of Contents
* [Building a landlab component](#Building-a-landlab-component)
  * [1. Linear Diffusion as a Function](#1.-Linear-Diffusion-as-a-Function)
  * [2. Represent the function as a class](#2.-Represent-the-function-as-a-class)
  * [3. Create a landlab component class](#3.-Create-a-landlab-component-class)
  * [4. Create a LandscapeUplifter component](#4.-Create-a-LandscapeUplifter-component)
  * [5. Couple LandscapeDiffuser with LandscapeUplifter](#5.-Couple-LandscapeDiffuser-with-LandscapeUplifter)
  * [6. Construct a landscape evolution model](#6.-Construct-a-landscape-evolution-model)

# Building a landlab component

In this next section we will build two landlab components: one with my help and one on your own. These two components will model physical processes that we will use later on to build a landscape evolution model.

The first component will be a linear diffusion component to model soil creep on hillslopes and the second will uplift (or subside) the landscape through time.

In [ ]:
from landlab import LinkStatus, NodeStatus, RasterModelGrid
from tqdm import trange

## 1. Linear Diffusion as a Function

We will begin with a function that solves the diffusion equation on a landlab grid using the equations from the previous section. This will be the function we will turn into a *landlab* component.

Our function takes a *landlab* grid, an array of values to diffuse, and a keyword argument to specify the diffusion coeffiecient. The result will be the diffusion rate at each node of the grid.

In [ ]:
def calc_diffusion_rate(grid, value_at_node, diffusion_coefficient=1.0):
    """Calculate the rate of diffusion of a quantity defined on a landlab grid.

    Parameters
    ----------
    grid : ModelGrid
        A landlab grid.
    value_at_node : ndarray of float
        The quantity to diffuse, defined at the grid's nodes.
    diffusion_coefficient : float, optional
        Diffusion coefficent to use for the diffusion.

    Returns
    -------
    value_at_node
        Input array of values after diffusion.
    """
    qs = -diffusion_coefficient * grid.calc_grad_at_link(value_at_node)
    qs[grid.status_at_link != LinkStatus.ACTIVE] = 0.0

    dzdt = -grid.calc_flux_div_at_node(qs)
    return dzdt

Now try to run this function on a *landlab* grid to diffuse some quantity. For simplicity, use a `RasterModelGrid`. Although `calc_diffusion_rate` function is agnostic as to what grid you pass it, it's will be easier to visualize a raster grid.

To begin with, see if you can,
* Create a `RasterModelGrid`
* Create an array of values (at grid nodes) to diffuse
* Set boundary conditions

To make things interesting, you will need something to diffuse. Try to initialize your array of values with a step that goes from 0.0 for $x <= 100$ to 10.0 for $x > 100.0$.

In [ ]:
# Type your code here

In [ ]:
grid = RasterModelGrid((100, 200))

z = grid.zeros(at="node")
z[(grid.x_of_node > 100)] = 10.0

grid.status_at_node[grid.nodes_at_left_edge] = NodeStatus.FIXED_VALUE
grid.status_at_node[grid.nodes_at_right_edge] = NodeStatus.FIXED_VALUE
grid.status_at_node[grid.nodes_at_top_edge] = NodeStatus.CLOSED
grid.status_at_node[grid.nodes_at_bottom_edge] = NodeStatus.CLOSED

grid.imshow(z)

Before we start to run diffusion through time, we need to determine a stable time step so our solution doesn't blow up.

We will use a timestep with a [Courant–Friedrichs–Lewy condition](https://en.wikipedia.org/wiki/Courant–Friedrichs–Lewy_condition) of $C_{cfl}=0.2$. This will keep our solution numerically stable. 

$C_{cfl} = \frac{\Delta t D}{\Delta x^2} = 0.2$

Write a function that calculates a stable time step, $dt$.

In [ ]:
def calc_stable_time_step(grid, diffusion_coefficient):
    return  # Type your code here

In [ ]:
def calc_stable_time_step(grid, diffusion_coefficient):
    return 0.2 * grid.length_of_link.min() ** 2 / diffusion_coefficient

And now for some landform evolution! First calculate a stable time step for your simulation and then write a 1000 iteration loop that, for each iteration, calculates the diffusion rate and then adds that to the current landscape.

If you like, you can experiment with different values for the diffusion coefficient and the time step.

In [ ]:
# Type your code here: calculate time step

for _ in trange(1000):
    ...  # Type your code here: calculate diffusion and update *z*

grid.imshow(z)

In [ ]:
diffusion_coefficient = 1.0
dt = calc_stable_time_step(grid, diffusion_coefficient)

for _ in trange(1000):
    z += calc_diffusion_rate(grid, z, diffusion_coefficient=diffusion_coefficient) * dt

grid.imshow(z)

## 2. Represent the function as a class

The next step in turning a function into a component is to represent your function as a class. To begin with, we implement two methods:
* `__init__`: this method is called when the class is instantiated and will accept to arguments. The first MUST be a landlab grid, the second will be a keyword that sets the diffusion coefficient.
* `calc_rate`: this method will take an array of values and return the diffusion rate at each element.
* `calc_stable_time_step`: this method will calculate a stable time step for the diffuser.

So that we're all on the same page, let's call the new class `LandscapeDiffuser`.

In [ ]:
# Type your code here

In [ ]:
class LandscapeDiffuser:
    def __init__(self, grid, diffusion_coefficient=1.0):
        self._grid = grid
        self._diffusion_coefficient = diffusion_coefficient

    def calc_rate(self, values):
        return calc_diffusion_rate(
            self._grid, values, diffusion_coefficient=self._diffusion_coefficient
        )

    def calc_stable_time_step(self):
        return 0.2 * self._grid.length_of_link.min() ** 2 / self._diffusion_coefficient

Now, just as you did before, run diffusion on a *landlab* grid only this time using the new `LandscapeDiffuser` class.

As before, create a grid, a quantity to diffuse, and set boundary conditions.

In [ ]:
grid = RasterModelGrid((100, 200))

z = grid.add_zeros("topographic__elevation", at="node")
z[(grid.x_of_node > 100) & (grid.y_of_node > 50)] = 10.0

grid.status_at_node[grid.nodes_at_left_edge] = NodeStatus.CLOSED
grid.status_at_node[grid.nodes_at_right_edge] = NodeStatus.CLOSED
grid.status_at_node[grid.nodes_at_top_edge] = NodeStatus.FIXED_VALUE
grid.status_at_node[grid.nodes_at_bottom_edge] = NodeStatus.CLOSED

grid.imshow(z)

Instantiate the `LandscapeDiffuser` and, again, run it over successive time steps.

In [ ]:
# Type your code here

In [ ]:
diffusion_coefficient = 0.5
diffuse = LandscapeDiffuser(grid, diffusion_coefficient=diffusion_coefficient)
dt = diffuse.calc_stable_time_step()

for _ in trange(10000):
    z += diffuse.calc_rate(z) * dt

grid.imshow(z)

## 3. Create a landlab component class

We need to add just a few more things to the `LandscapeDiffuser` before we can call it a complete *landlab* component.

* All components MUST inherit from `landlab.Component`
* All components MUST have a `run_one_step` function
* All components MUST call `super().__init__(grid)` as part of its `__init__` method
* All components MUST have two attributes:
  * `_info` is a dictionary that provides information about a component's input and output fields
  * `_unit_agnostic` is a boolean that indicates for input and output fields MUST be given in specific units.

The `run_one_step` method acts on a *landlab* field and can accept an optional *dt* keyword that tells it how long to run for. For the `LandscapeDiffuser` let's have it operate on a field called *topographic__elevation* that's defined at *nodes*.

The `_info` attribute describes each field the component uses. It is a dictionary where keys are field names and values are descriptions. For example, you can look at the `_info` field of an existing component.

In [ ]:
from landlab.components import BedrockLandslider

BedrockLandslider._info

Attributes for each field include,
* *dtype*: the data type of the field (float, int, etc.)
* *intent*: if the field is an input ('in') output ('out') or both ('inout')
* *optional*: boolean to indicate if the field is required
* *unit*: the units of the field as a string
* *mapping*: on what grid element the quantity is defined ('node', 'cell', 'link', etc.)
* *doc*: a short description of the field

Fill in the template below with the *landlab* component requirements.

In [ ]:
from landlab import Component


class LandscapeDiffuser(Component):
    _info = {
        # Add description of input/output fields here
    }

    _unit_agnostic = True or False  # Should this component be unit agnostic or not?

    def __init__(self, grid, diffusion_coefficient=1.0):
        self._grid = grid
        self._diffusion_coefficient = diffusion_coefficient

        super().__init__(grid)

    def calc_rate(self, values):
        return calc_diffusion_rate(
            self._grid, values, diffusion_coefficient=self._diffusion_coefficient
        )

    def calc_stable_time_step(self):
        return 0.2 * self._grid.length_of_link.min() ** 2 / self._diffusion_coefficient

    def run_one_step(self, dt=1.0):
        """Run diffusion on *topographic__elevation*"""
        # Type your code here.

In [ ]:
from landlab import Component


class LandscapeDiffuser(Component):
    _info = {
        "topographic__elevation": {
            "dtype": float,
            "intent": "inout",
            "optional": False,
            "units": "m",
            "mapping": "node",
            "doc": "Diffuse sediment over a landscape",
        },
    }

    _unit_agnostic = True

    def __init__(self, grid, diffusion_coefficient=1.0):
        self._grid = grid
        self._diffusion_coefficient = diffusion_coefficient

        super().__init__(grid)

    def calc_rate(self, values):
        return calc_diffusion_rate(
            self._grid, values, diffusion_coefficient=self._diffusion_coefficient
        )

    def calc_stable_time_step(self):
        return 0.2 * self._grid.length_of_link.min() ** 2 / self._diffusion_coefficient

    def run_one_step(self, dt=1.0):
        time_step = self.calc_stable_time_step()
        n_steps = int(dt / time_step)
        for _ in range(n_steps):
            dzdt = self.calc_rate(grid.at_node["topographic__elevation"])
            grid.at_node["topographic__elevation"] += dzdt * time_step

        remaining = dt - (n_steps * time_step)
        if remaining > 0:
            dzdt = self.calc_rate(grid.at_node["topographic__elevation"])
            grid.at_node["topographic__elevation"] += dzdt * remaining

Now run the component as before, this time using the `run_one_step` method and ***adding a topographic__elevation field***.

In [ ]:
grid = RasterModelGrid((100, 200))

z = grid.add_zeros("topographic__elevation", at="node")
z[(grid.x_of_node > 100) & (grid.y_of_node > 50)] = 10.0

grid.status_at_node[grid.nodes_at_left_edge] = NodeStatus.FIXED_VALUE
grid.status_at_node[grid.nodes_at_right_edge] = NodeStatus.CLOSED
grid.status_at_node[grid.nodes_at_top_edge] = NodeStatus.CLOSED
grid.status_at_node[grid.nodes_at_bottom_edge] = NodeStatus.CLOSED

grid.imshow(grid.at_node["topographic__elevation"])

Once again, write some code that will run the diffuser over the grid some number of times. Let's say for $1000$ time steps with a $dt=0.2$.

In [ ]:
# Type your code here.

In [ ]:
diffuse = LandscapeDiffuser(grid, diffusion_coefficient=0.5)
for _ in trange(1000):
    diffuse.run_one_step(dt=0.2)
grid.imshow(grid.at_node["topographic__elevation"])

## 4. Create a LandscapeUplifter component


Now create a `LandscapeUplifter` component based on a function that uplifts and landscape at a constant rate.

To be compatible with the `LandscapeDiffuser`, have the uplifter also act on a field called *topographic__elevation* defined at nodes.

In [ ]:
# Type your code here

In [ ]:
class LandscapeUplifter(Component):
    _info = {
        "topographic__elevation": {
            "dtype": float,
            "intent": "inout",
            "optional": False,
            "units": "m",
            "mapping": "node",
            "doc": "Elevation of a landspace surface",
        },
    }

    _unit_agnostic = True

    def __init__(self, grid, uplift_rate=0.0):
        self._uplift_rate = uplift_rate
        super().__init__(grid)

    def run_one_step(self, dt=1.0):
        self.grid.at_node["topographic__elevation"][self.grid.core_nodes] += (
            self._uplift_rate * dt
        )

In [ ]:
grid = RasterModelGrid((100, 200))

z = grid.add_zeros("topographic__elevation", at="node")

grid.status_at_node[grid.nodes_at_left_edge] = NodeStatus.FIXED_VALUE
grid.status_at_node[grid.nodes_at_right_edge] = NodeStatus.FIXED_VALUE
grid.status_at_node[grid.nodes_at_top_edge] = NodeStatus.CLOSED
grid.status_at_node[grid.nodes_at_bottom_edge] = NodeStatus.CLOSED

grid.imshow(grid.at_node["topographic__elevation"])

As with the `LandscapeDiffuser` try writing code to run your `LandscapeUplifter` within a time loop.

In [ ]:
# Type your code here

In [ ]:
uplift = LandscapeUplifter(grid, uplift_rate=0.001)

dt = 1000.0
for _ in trange(20):
    uplift.run_one_step(dt=dt)

grid.imshow(grid.at_node["topographic__elevation"])

## 5. Couple LandscapeDiffuser with LandscapeUplifter

To couple the `LandscapeDiffuser` and the `LandscapeUplifter` we first instantiate both components *with the same grid*. Within the time loop, we call one component's *run_one_step* method, and then the other's.

Try writing the code to couple these two components.

In [ ]:
# Type your code here

In [ ]:
diffuse = LandscapeDiffuser(grid, diffusion_coefficient=0.2)
uplift = LandscapeUplifter(grid, uplift_rate=0.001)

dt = 1000.0
for _ in trange(20):
    uplift.run_one_step(dt=dt)
    diffuse.run_one_step(dt=dt)

grid.imshow(grid.at_node["topographic__elevation"])

## 6. Construct a landscape evolution model

In [ ]:
from landlab.components import FastscapeEroder, FlowAccumulator
from numpy.random import default_rng

rng = default_rng()

Create an initial grid that's pretty much flat except for a small amount of noise. If you like, feel free to play around with the size of the grid, the grid spacing, or the boundary conditions.

In [ ]:
# Type your code here

In [ ]:
grid = RasterModelGrid((100, 200), xy_spacing=0.02)

z = grid.add_field(
    "topographic__elevation",
    rng.random(grid.number_of_nodes) * 1e-5,
    at="node",
)

grid.status_at_node[grid.nodes_at_left_edge] = NodeStatus.CLOSED
grid.status_at_node[grid.nodes_at_right_edge] = NodeStatus.CLOSED
grid.status_at_node[grid.nodes_at_top_edge] = NodeStatus.FIXED_VALUE
grid.status_at_node[grid.nodes_at_bottom_edge] = NodeStatus.FIXED_VALUE

grid.imshow(grid.at_node["topographic__elevation"])

Our LEM will couple our `LandscapeDiffuser` and `LandscapeUplifter` along with two new components from the *landlab* component library: `FlowAccumulator` and `FastscapeEroder`. The `FlowAccumulator` will route flow over the landscape and the `FastscapeEroder` will erode and transport sediment over the landscape.

Step 1: Instantiate all of the components.

In [ ]:
# Type your code here

In [ ]:
accumulate = FlowAccumulator(grid)
erode = FastscapeEroder(grid, K_sp=0.3, m_sp=0.5)
diffuse = LandscapeDiffuser(grid, diffusion_coefficient=0.0004)
uplift = LandscapeUplifter(grid, uplift_rate=0.001)

Step 2: Run each component within a time loop

In [ ]:
# Type your code here

In [ ]:
dt = 0.5
for _ in trange(100):
    uplift.run_one_step(dt=dt)
    diffuse.run_one_step(dt=dt)
    accumulate.run_one_step()
    erode.run_one_step(dt=dt)

grid.imshow(grid.at_node["topographic__elevation"])